# 使用Python批量下载哨兵一号（sentinel-1）的精密轨道数据

## 1. 安装Python及data_downloader包
本文使用的是Python脚本，来批量下载文件。如果使用本文的下载方式，请先下载Python3.6以上的版本，并使用pip安装data_downloader包:

```bash
pip install data_downloader
```

本教程使用从AFS下载精密轨道数据需要NASA的授权，所以需要在`.netrc`中加入登录信息，这样下载的时候就不会因登录信息而被拒绝访问。

将下面代码中的`your_username`与`your_password`，改为自己在哨兵官网注册的用户名与密码，并复制到Python编辑器中执行。


In [ ]:
from data_downloader import downloader,parse_urls
from pathlib import Path
import pandas as pd
import datetime as dt

netrc = downloader.Netrc()
netrc.add('urs.earthdata.nasa.gov', 'your_username', 'your_password')

## 2. 下载精密轨道数据与辅助数据
哨兵一号的精密轨道数据（aux_poeorb, Precise orbit）与辅助数据(aux_cal,  Auxiliary data) 可以从欧空局与AFS下载。由于欧空局这两年换了几次下载链接，本教程采用的是从AFS上下载。

复制并运行下面代码即可下载。

In [ ]:
# 指定aux_cal，辅助数据 的下载文件夹
folder_aux = Path('/media/data/aux')  

# 指定aux_poeorb，精密轨道数据 的下载文件夹
folder_preorb = Path('/media/data/poeorb') 


def filter_aux_cal_urls(urls, platform='all'):
    '''filter files from urls of aux_cal by platform.
    
    Parameters:
    ----------
    urls : list
        a list contains the urls of aux_cal 
    platform : str
        platform of satellite. should be one of ['S1A', 'S1B','all']  
    '''
    if platform in ['S1A', 'S1B','all']:
        if platform == 'all':
            platform = ['S1A', 'S1B']
        else:
            platform = [platform]
    else:
        raise ValueError("platform must be one of ['S1A', 'S1B','all']")
    
    _urls = [i for i in urls if Path(i).suffix == '.SAFE']
    urls_filter = [i for i in _urls if Path(i).stem[:3] in platform]
    
    return urls_filter
            

def filter_aux_poeorb_urls(urls, date_start, date_end, platform='all'):
    '''filter files from urls of aux_poeorb(precise orbit) by date and platform.
    
    Parameters:
    ----------
    urls : list
        a list contains the urls of aux_cal
    date_start, date_end : str
    	start/end date to filter
    platform : str
        platform of satellite. should be one of ['S1A', 'S1B','all']  
    '''
    if platform in ['S1A', 'S1B', 'all']:
        if platform == 'all':
            platform = ['S1A', 'S1B']
        else:
            platform = [platform]
    else:
        raise ValueError("platform must be one of ['S1A', 'S1B','all']")

    date_start = pd.to_datetime(date_start).date()
    date_end = pd.to_datetime(date_end).date()
    
    _urls = [i for i in urls if Path(i).suffix == '.EOF']
    urls_filter = []
    for i in _urls:
        name = Path(i).stem
        dt_i = (pd.to_datetime(name.split('_')[-1]).date() 
                - dt.timedelta(days=1))
        
        if (name[:3] in platform and 
            date_start <= dt_i <= date_end):
            urls_filter.append(i)
        
    return urls_filter

					# 执行下载 #
# ########### download aux_cal  #####################
home_aux_cal = 'https://s1qc.asf.alaska.edu/aux_cal/'
urls_aux_cal = parse_urls.from_html(home_aux_cal)
urls = filter_aux_cal_urls(urls_aux_cal,'S1A') # 获取S1A的所有辅助数据的链接
downloader.async_download_datas(urls, folder=folder_aux)


########### download precise orbit ############
home_preorb = 'https://s1qc.asf.alaska.edu/aux_poeorb/'
urls_preorb = parse_urls.from_html(home_preorb)
urls = filter_aux_poeorb_urls(urls_preorb, 
                              '20210101', '20220301',
                              'S1A') # 获取所有S1A在20210101-20220301期间的精密轨道数据的链接
downloader.download_datas(urls, folder=folder_preorb)